In [1]:
from core.utils import Tibanna
from core import ff_utils
from core.utils import run_workflow
from datetime import datetime
from core.wfr import *

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
    

In [2]:
import time

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

wf_dict =[
            {'wf_name': 'pairsqc-single',
             'wf_uuid': 'ae3a87cb-3fa2-469e-97c7-540fc2d0a117',
             'parameters':{"enzyme": "4",
                           "sample_name": "4DNFILXNOYC7"}    
            }    
          ]


add_wfr = False

# url for hic exps
pairs_url = "/search/?file_format=pairs" + \
                    "&type=FileProcessed" + \
                    "&status=released&status=released%20to%20project&status=uploaded" + \
                    "&workflow_run_outputs.workflow.title=Hi-C%20Post-alignment%20Processing" + \
                    "&workflow_run_outputs.workflow.title=Generation%20of%20multiresolution%20Hi-C%20contact%20matrices%20from%20a%20set%20of%20contact%20lists"
pairs_files = ff_utils.get_metadata(pairs_url , connection=ff)['@graph']
pairs_files = [i for i in pairs_files if not i.get('quality_metric')]

counter = 0
completed = 0
print len(pairs_files)

okay = 0
non = 0
running = 0
for a_pairs in pairs_files:
    
#     if a_pairs['accession'] == '4DNFIYECESRC':
#         print(a_pairs['accession'], "still running - manual")
#         continue
    report = get_wfr_out(a_pairs['accession'], 'pairsqc-single 0.2.5', None, ff)
    if report.startswith('no') or not report:
        non += 1
        print(a_pairs['accession'], 'needs a run')
        if add_wfr:
            # get experiment info
            exp_acc = a_pairs.get('source_experiments')
            if not exp_acc:
                print(a_pairs['accession'], 'does not have source experiments, skipping')
                continue
            nz_num, chrsize = extract_nz_file(exp_acc[0], ff)
            if not nz_num:
                print(a_pairs['accession'], 'does not have RE size, skipping')
                continue
                
            parameters = {"enzyme": nz_num,
                          "sample_name": a_pairs['accession']} 
            
            run_guide = {'wf_name': 'pairsqc-single',
                         'wf_uuid': 'ae3a87cb-3fa2-469e-97c7-540fc2d0a117',
                         'parameters': parameters}
            
            inp_f = {'input_pairs':a_pairs['accession'], 'chromsizes':chrsize} 
            run_missing_wfr(run_guide, inp_f, a_pairs['accession'], ff, env, tibanna)

            
    elif report == 'running':
        running += 1
        print(a_pairs['accession'], 'still running')
    else:
        okay += 1
        
print("okay, not okay, running")    
print(okay, non, running)


44
(u'4DNFIB13H9QZ', 'needs a run')
(u'4DNFIT3RD5VO', 'needs a run')
(u'4DNFI155DWSK', 'needs a run')
72.7321788483
(u'4DNFI54UUBPY', 'still running')
72.7312823467
(u'4DNFITHXDLLC', 'still running')
72.7303615458
(u'4DNFISMHCSSH', 'still running')
(u'4DNFI5DDBY89', 'needs a run')
72.7289869189
(u'4DNFI37N5TCD', 'still running')
(u'4DNFIL9C2NVL', 'needs a run')
72.7276621069
(u'4DNFIX9FYTTA', 'still running')
(u'4DNFIS6ISGW7', 'needs a run')
(u'4DNFI8L2GKFO', 'needs a run')
(u'4DNFIYOUXDA5', 'needs a run')
(u'4DNFI9U9NJ89', 'needs a run')
(u'4DNFIKP2V6HQ', 'needs a run')
(u'4DNFI6ETC1JB', 'needs a run')
72.7207475483
(u'4DNFIT57JYMU', 'still running')
72.7193404475
(u'4DNFIHTXJC8B', 'still running')
72.7181211958
(u'4DNFI224W9FM', 'still running')
72.7165243308
(u'4DNFIHCQLL2H', 'still running')
72.7155862886
(u'4DNFILWV3C1I', 'still running')
72.7140092478
(u'4DNFIQXWVVB6', 'still running')
(u'4DNFI3O8M6SZ', 'needs a run')
72.7108342042
(u'4DNFI2HQA2EE', 'still running')
72.709734435


In [ ]:
# Release pairs qc

pairs_url = '/search/?file_format=pairs&status=released&status=released%20to%20project&status=uploaded&type=FileProcessed'
pairs_files = ff_utils.get_metadata(pairs_url , connection=ff)['@graph']
pairs_files = [i for i in pairs_files if i.get('quality_metric')]
print len(pairs_files)
success = 0
patch_problem = 0

for pairs in pairs_files:
    pairs_status = pairs['status']
    if pairs_status in ['released', 'released to project']:
        qc_status = ff_utils.get_metadata(pairs['quality_metric'],connection=ff)['status']
        if qc_status != pairs_status:
            patch_data = {"status": pairs_status}
            try:
                ff_utils.patch_metadata(patch_data, obj_id=pairs['quality_metric'] ,connection=ff)
                success += 1
                print success
            except Exception as e:
                print e
                print
                patch_problem += 1
print success
print patch_problem

